<a href="https://colab.research.google.com/github/mostafa202020/CCDMR/blob/main/CCDMR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install matplotlib
!pip install sktime
!pip install xgboost
!pip install javalang
!pip install openpyxl xlwt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!cp  /content/drive/MyDrive/bcb.rar  /content/
!apt-get install -y unrar

In [ ]:
!cp  /content/drive/MyDrive/clonedatasetfinal/merged_with_T1.csv  /content/
!cp  /content/drive/MyDrive/clonedatasetfinal/merged_with_T4.csv  /content/
!cp  /content/drive/MyDrive/clonedatasetfinal/merged_with_ST3.csv  /content/
!cp  /content/drive/MyDrive/clonedatasetfinal/merged_with_T2.csv  /content/
!cp  /content/drive/MyDrive/clonedatasetfinal/merged_with_MT3.csv  /content/

In [ ]:
import os

# Replace 'your_file.rar' with the name of your uploaded RAR file
rar_file_path ='/content/bcb.rar'# '/content/id2sourcecode.rar'
extraction_path = '/content/java_files/'

# Create the extraction directory if it doesn't exist
os.makedirs(extraction_path, exist_ok=True)

# Extract the RAR file
!unrar x -y {rar_file_path} {extraction_path}

print(f"Files extracted to {extraction_path}")

In [ ]:
from google.colab import drive
import os

# Mount Google Drive
#drive.mount('/content/drive')

def count_files_in_directory(directory_path):
    total_files = 0
    for root, dirs, files in os.walk(directory_path):
        total_files += len(files)
    return total_files

# Directory to count files in
directory_path = '/content/java_files'


# Count files
num_files = count_files_in_directory(directory_path)
print(f"Total number of files in directory '{directory_path}': {num_files}")

Total number of files in directory '/content/java_files': 0


In [ ]:
#java's to sequences
import os

import glob
import javalang

import os
import glob
import javalang
from google.colab import drive
drive.mount('/content/drive')
token_dict = {}
token_dictt = {}
def tokenize_code(code):
    global token_dict
    global token_dictt

    try:
        tokens = list(javalang.tokenizer.tokenize(code))
    except javalang.tokenizer.LexerError as e:
        print(f"LexerError: {e}")
        print("The problematic code segment is likely to have an unterminated character/string literal or other lexical error.")
        return []

    token_sequence = []

    for token in tokens:
        token_value = token.value
        token_type = str(type(token))[:-2].split(".")[-1]

        if isinstance(token, javalang.tokenizer.Identifier):
            token_value = 'id'
        if isinstance(token, javalang.tokenizer.String):
            token_value = 'string'

        if token_value not in token_dict:
            token_dict[token_value] = len(token_dict) + 1  # Start token numbering from 1

        if token_type not in token_dictt:
            token_dictt[token_type] = len(token_dictt) + 1

        token_sequence.append(token_dict[token_value])
        token_sequence.append(token_dictt[token_type])


    return token_sequence
import re
import re

def remove_comments(source_code):

    # Remove block comments
    source_code = re.sub(r'/\*.*?\*/', '', source_code, flags=re.DOTALL)
    # Remove line comments
    source_code = re.sub(r'//.*', '', source_code)
    return source_code

def clean_source_code(source_code):

    return source_code.encode('ascii', 'ignore').decode('ascii')
def getCodeBlock(file_path):
    global token_dict
    global token_dictt

    block = []
    # print(file_path)
    try:
        with open(file_path, 'r', encoding='utf-8') as temp_file:
            lines = temp_file.readlines()
            for line in lines:
                try:
                    tokens = list(javalang.tokenizer.tokenize(line))
                    for token in tokens:
                        token_value = token.value
                        token_type = str(type(token))[:-2].split(".")[-1]

                        if isinstance(token, javalang.tokenizer.Identifier):
                                               token_value = 'id'
                        if isinstance(token, javalang.tokenizer.String):
                                               token_value = 'string'

                        if token_value not in token_dict:
                                 token_dict[token_value] = len(token_dict) + 1  # Start token numbering from 1

                        if token_type not in token_dictt:
                                  token_dictt[token_type] = len(token_dictt) + 1

                        #block.append(token_dict[token_value])
                        block.append(token_dictt[token_type])


                except javalang.tokenizer.LexerError as e:
                    print(f"LexerError in file {file_path} at line: {line.strip()}: {e}")
                    # Optionally handle the line with the error, e.g., skip it
                    continue
    except UnicodeDecodeError as e:
        print(f"Error reading file {file_path}: {e}")
        return None
    return block

def process_java_files(input_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    java_files = glob.glob(os.path.join(input_dir, '*.java'))
      # Initialize the dictionary outside the loop

    for java_file in java_files:

        token_sequence = getCodeBlock(java_file)
        token_sequence_str = ' '.join(map(str, token_sequence))

        base_filename = os.path.basename(java_file)
        output_filename = os.path.splitext(base_filename)[0] + '.txt'
        output_file_path = os.path.join(output_dir, output_filename)

        with open(output_file_path, 'w', encoding='utf-8') as f:
            f.write(token_sequence_str)
            #print(token_sequence_str)


# Example usage
input_directory = '/content/java_files'  # Replace with the path to the directory containing Java files
output_directory = '/content/sequencessall'  # Replace with the path to the directory where you want to save the output
  # Replace with the path to the directory where you want to save the output

process_java_files(input_directory, output_directory)

In [ ]:
#fianl version with allclassifiers here for test 3dnumpy and multii
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.svm import SVC
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import precision_recall_curve, roc_curve, auc, accuracy_score, classification_report, f1_score
from sktime.transformations.panel.rocket import Rocket,MiniRocket
from sktime.datatypes._panel._convert import from_2d_array_to_nested
import time
import pandas as pd
import numpy as np
import os
from google.colab import drive
drive.mount('/content/drive')
#make a change of parameters
clone='MT3'
cl='MT3'
pathcsv ='/content/drive/MyDrive/clonedatasetfinal/merged_with_'+str(cl)+'.csv'#'/content/MT3_allinone.csv'# '/content/drive/MyDrive/id2sourcecode/sequencesall'
#pathcsv = 'D:/BCB/astn/clone_4.csv'
time_series_dir = '/content/sequencessall'
rl=1#step
num_kernels_list = [500,1000,10000]
def count_numbers_in_file(file_path):
    with open(file_path, 'r') as file:
        content = file.read()
    numbers = content.split()
    number_count = len(numbers)
    return number_count

# Function to get the length of a time series
def get_time_series_length(file):
    if not os.path.exists(file):
        return 0
    ts = np.loadtxt(file)
    return len(ts)
import os
import numpy as np
from sklearn.tree import DecisionTreeClassifier


def load_and_merge_time_series(file1, file2, max_length):
    if not os.path.exists(file1):
        print(f"File not found: {file1}")
        return None
    if not os.path.exists(file2):
        print(f"File not found: {file2}")
        return None

    ts1 = np.loadtxt(file1)
    ts2 = np.loadtxt(file2)
    concatenated_ts = np.concatenate((ts1, ts2))
    concatenated_ts  = np.pad( concatenated_ts , (0, 2*max_length - len( concatenated_ts )), mode='constant')
    """


    ts1 = np.pad(ts1, (0, max_length - len(ts1)), mode='constant')
    ts2 = np.pad(ts2, (0, max_length - len(ts2)), mode='constant')
    """







    #differential_encoded_ts = np.diff(concatenated_ts, prepend=0)
    return  concatenated_ts

import os
import numpy as np
import hashlib
import time

# Function to evaluate classifiers
def evaluate_classifiers(X_train, X_test, y_train, y_test, classifiers):
    results = {}
    for name, clf in classifiers.items():
        start_time = time.time()
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        report = classification_report(y_test, y_pred, target_names=['Class 0', 'Class 1'])
        y_scores = clf.predict_proba(X_test)[:, 1] if hasattr(clf, "predict_proba") else clf.decision_function(X_test)
        precision, recall, _ = precision_recall_curve(y_test, y_scores)
        fpr, tpr, _ = roc_curve(y_test, y_scores)
        roc_auc = auc(fpr, tpr)
        end_time = time.time()
        results[name] = {
            'accuracy': accuracy,
            'f1_score': f1,
            'report': report,
            'precision': precision,
            'recall': recall,
            'fpr': fpr,
            'tpr': tpr,
            'roc_auc': roc_auc,
            'time': end_time - start_time
        }
    return results

# Path to CSV file and directory containing time series files



# Load CSV with file names and class labels
df_labels = pd.read_csv(pathcsv)

# Determine the maximum length of time series

max_length = 0

min_length=0
av=0
l=0
for i in range(0, len(df_labels), rl):
    row = df_labels.iloc[i]
    """
    file1 = os.path.join(time_series_dir, str(row['id1']).strip() + '.txt')
    file2 = os.path.join(time_series_dir, str(row['id2']).strip() + '.txt')
    """
    file1 = os.path.join(time_series_dir, str(row['FunID1']).strip() + '.txt')
    file2 = os.path.join(time_series_dir, str(row['FunID2']).strip() + '.txt')

    ts1_length = get_time_series_length(file1)
    ts2_length = get_time_series_length(file2)

    max_length = max(max_length, ts1_length, ts2_length)
    min_length = min(min_length, ts1_length, ts2_length)
    #av=(av+(ts1_length+ts1_length)/2)/2
    l=l+2
max_length=(max_length)
print("max length nhnfgfor ",max_length, "min=",min, "av",av/l)
#max_length=count+3#(count//2)+3


# Extract features and labels
features = []
labels = []

for i in range(0, len(df_labels), rl):
    row = df_labels.iloc[i]
    """
    file1 = os.path.join(time_series_dir, str(row['id1']).strip() + '.txt')
    file2 = os.path.join(time_series_dir, str(row['id2']).strip() + '.txt')
    label = row['label']
    """
    file1 = os.path.join(time_series_dir, str(row['FunID1']).strip() + '.txt')
    file2 = os.path.join(time_series_dir, str(row['FunID2']).strip() + '.txt')
    label = row['Type']

    merged_ts = load_and_merge_time_series(file1, file2, max_length)

    #print("mergerd is herer",merged_ts)
    if merged_ts is not None:
        features.append(merged_ts)
        labels.append(label)
        #print("rrrrererer")

if not features:
    raise ValueError("No valid time series data found.")

# Convert lists to numpy arrays
time_series_array = np.array(features)

# Create a MultiIndex with the series index and time step index
index = pd.MultiIndex.from_product(
    [range(time_series_array.shape[0]), range(time_series_array.shape[1])],
    names=['series', 'time_step']
)

# Flatten the array to fit into the DataFrame
flattened_array = time_series_array.flatten()

# Create the DataFrame with the MultiIndex
df_multiindex = pd.DataFrame(flattened_array, index=index, columns=["value"])

print(df_multiindex)

# Plot the third time series (index 2 since Python uses 0-based indexing)
#df_multiindex['series'].iloc[2].plot()

# Add labels and title
plt.xlabel('Time')
plt.ylabel('Value')
plt.title('Time Series ')

# Show the plot
plt.show()
print(labels)

from sktime.datatypes import check_raise
#from sktime.datatypes import check_is_pd-multiindex



features=df_multiindex
print("checking if features is conform to multiindex")
print(check_raise(features, mtype="pd-multiindex"))
#check_is_pd-multiindex(features, mtype="pd-multiindex", return_metadata=True)
labels = np.array(labels)
labels = np.where(labels == cl, 1, 0)
print(labels)
import numpy as np
import matplotlib.pyplot as plt

# Assuming labels is your array and cl is the condition


# Count the number of 1s and 0s
unique, counts = np.unique(labels, return_counts=True)
count_dict = dict(zip(unique, counts))
for class_label, count in count_dict.items():
    print(f"Class {class_label}: {count} instances")

# Create a bar chart
plt.bar(count_dict.keys(), count_dict.values(), color=['blue', 'orange'])
plt.xlabel('Label')
plt.ylabel('Count')
plt.title('Number of 1s and 0s in labels')
plt.xticks([0, 1], ['0', '1'])
plt.show()


unique=0
counts =0
features_df=0
print(len(features))
# List of classifiers
"""
Logistic Regression': LogisticRegression(max_iter=100),
    'XGBoost': xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
    'Random Forest': RandomForestClassifier(n_estimators=100),
    'Naive Bayes': GaussianNB(),
    'Decision Tree': DecisionTreeClassifier(),
    'SVM': SVC()}
"""
from sklearn.svm import SVC, LinearSVC
classifiers = {
    'Logistic Regression': LogisticRegression(max_iter=100),
    'XGBoost': xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
    'Random Forest': RandomForestClassifier(n_estimators=100),
    'Naive Bayes': GaussianNB(),
    'Decision Tree': DecisionTreeClassifier(),

    }






# Vary number of kernels and evaluate
#,1150,1300]#[600, 500,700 ,900]
all_results = {}
print("training")
import joblib
from sklearn.svm import SVC, LinearSVC
# Assuming trf is your MiniRocket instance

from sklearn.metrics import precision_score, recall_score, f1_score
for num_kernels in num_kernels_list:
    print(f"\nTraining MiniRocket with {num_kernels} kernels...")
    trf_start_time = time.time()
    trf = MiniRocket(num_kernels=num_kernels)
    trf.fit(features)
    trf_end_time = time.time()
    trf_training_time = trf_end_time - trf_start_time
    joblib.dump(trf, f'/content/drive/MyDrive/mT3ker600mini_rocket_model{num_kernels}T4.pkl')
    trftr_start_time = time.time()
    transformed_features = trf.transform(features)
    trftr_end_time = time.time()

    X_train, X_test, y_train, y_test = train_test_split(transformed_features, labels, test_size=0.5, random_state=42, shuffle=True)

    print("shape xtr",X_train.shape)
    lrow =X_train[1]
    print(len(lrow), lrow)

    print("wtrain", len(X_train),len (X_train[0]),"=",len(y_train))
    print("wtest", len(X_test),len(X_test[0]) ,"=",len(y_train))
    print(y_train)
    print(y_test)



    results = {}
    for name, clf in classifiers.items():
        print(f"\nEvaluating {name}...")
        clf_start_time = time.time()
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        clf_end_time = time.time()

        if hasattr(clf, "decision_function"):
            y_scores = clf.decision_function(X_test)
        else:
            y_scores = clf.predict_proba(X_test)[:, 1]

        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        fpr, tpr, _ = roc_curve(y_test, y_scores)
        roc_auc = auc(fpr, tpr)

        results[name] = {
            'precision': precision,
            'recall': recall,
            'f1_score': f1_score(y_test, y_pred),
            'roc_auc': roc_auc,
            'inference_time': clf_end_time - clf_start_time,
            'training_time': trf_training_time,
            'transformation':trftr_end_time - trftr_start_time
        }

        # Print textual results
        print(f"Classifier: {name}")
        print(f"Precision: {precision.mean()}")
        print(f"Recall: {recall.mean()}")
        print(f"F1 Score: {results[name]['f1_score']}")
        print(f"ROC AUC: {results[name]['roc_auc']}")
        print(f"Inference Time: {results[name]['inference_time']} seconds")
        print(f"MiniRocket Training Time: {results[name]['training_time']} seconds")

    all_results[num_kernels] = results
    print('tran', trftr_end_time - trftr_start_time)

results_list = []
for num_kernels, results in all_results.items():
    for name, metrics in results.items():
        results_list.append({
            'Num Kernels': num_kernels,
            'Classifier': name,
            'Precision': metrics['precision'],
            'Recall': metrics['recall'],
            'F1 Score': metrics['f1_score'],
            'ROC AUC': metrics['roc_auc'],
            'Inference Time (s)': metrics['inference_time'],
            'MiniRocket Training Time (s)': metrics['training_time'],
            'MiniRocket Trnsformation Time (s)': metrics['transformation']
        })

df_results = pd.DataFrame(results_list)

# Print the DataFrame
print(df_results)
pth ='/content/drive/MyDrive/resultpaddafter/finalnewresults_'+str(cl)+'.csv'
df_results.to_csv(pth, index=False)
pthxls ='/content/drive/MyDrive/resultpaddafter/finalnewresults_'+str(cl)+'.xls'
df_results.to_excel(pthxls, index=False,engine='openpyxl')

In [ ]:
!pip install --upgrade tbb